# 〜スクレイピング〜

### J-LEAGUE-DATA-SITE

機械学習に使用するデータを集めていきます。  
Jリーグの公式HPから、試合日程、試合時の順位や得点数等をスクレイピングしていきます。  
https://data.j-league.or.jp/SFMS01/back

In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from IPython.display import Image
import time

まずは試合日程を取得します

In [4]:
team_id_list = ["14", "362", "269", "54", "270", "29", "271", "1", "94", "40", "35", "3", "27", "2", "11", "22", "4", "45", "21", "5", "34", "272", "6", "12", "273", "28", "46", "274", "78", "41", "275", "7", "13", "276", "347", "8", "39", "24", "9", "20", "18", "44", "42", "10", "330", "48", "36", "37", "369", "23", "43", "33", "47", "38", "31", "371", "338", "277", "339", "340", "341", "278"]
team_name_list = ["札幌", "八戸", "岩手", "仙台", "秋田", "山形", "福島", "鹿島", "水戸", "栃木", "群馬", "浦和", "大宮", "千葉", "柏", "FC東京", "東京Ｖ", "町田", "川崎Ｆ", "横浜FM", "横浜FC", "YS横浜", "横浜Ｆ", "湘南", "相模原", "甲府", "松本", "長野", "新潟", "富山", "金沢", "清水", "磐田", "藤枝", "沼津", "名古屋", "岐阜", "京都", "Ｇ大阪", "Ｃ大阪", "神戸", "鳥取", "岡山", "広島", "山口", "讃岐", "徳島", "愛媛", "今治", "福岡", "北九州", "鳥栖", "長崎", "熊本", "大分", "宮崎", "鹿児島", "琉球", "Ｆ東23", "Ｇ大23", "Ｃ大23", "J-22"]
url_head = "https://data.j-league.or.jp/SFMS01/search?team_ids="
url_foot = "&home_away_select=0&tv_relay_station_name="
df_result = None
for i in range(len(team_id_list)):
    
    url = url_head + team_id_list[i] + url_foot
    time.sleep(1) # スリープ処理
    j_data_list = pd.read_html(url)
    df_j_data = pd.DataFrame(j_data_list[0]) # データフレームに変換
    
    # チーム名のカラムを追加
    df_j_data["Team"] = team_name_list[i]
    
    df_result = pd.concat([df_result, df_j_data])

In [5]:
df_result

,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム,入場者数,インターネット中継・TV放送,Team
0,1997,ＹＮＣ 予選Ｂグループ,第１節第１日,03/08(土),15:01,Ｇ大阪,3-3,札幌,万博,10514.0,NaN,札幌
1,1997,ＹＮＣ 予選Ｂグループ,第２節第１日,03/15(土),16:02,札幌,2-2,Ｖ川崎,長崎,4823.0,NaN,札幌
2,1997,ＹＮＣ 予選Ｂグループ,第３節第１日,03/19(水),19:01,横浜M,1-3,札幌,三ツ沢,5121.0,NaN,札幌
3,1997,ＹＮＣ 予選Ｂグループ,第４節第１日,03/22(土),15:00,Ｖ川崎,2-2,札幌,等々力,6940.0,NaN,札幌
4,1997,ＹＮＣ 予選Ｂグループ,第５節第１日,03/26(水),19:00,札幌,1-2,横浜M,長良川,2913.0,NaN,札幌
...,...,...,...,...,...,...,...,...,...,...,...,...
64,2015,Ｊ３,第３４節第２日,10/18(日),13:05,富山,2-1,J-22,富山,2270.0,NaN,J-22
65,2015,Ｊ３,第３５節第１日,10/25(日),13:00,藤枝,1-4,J-22,エコパ,858.0,NaN,J-22
66,2015,Ｊ３,第３６節第２日,11/01(日),13:04,長野,2-1,J-22,南長野,4163.0,信越放送,J-22
67,2015,Ｊ３,第３７節第１日,11/08(日),13:00,福島,3-0,J-22,とうスタ,791.0,NaN,J-22


In [7]:
df_result.to_csv("match_schedule_original.csv")

次に、試合時のチーム情報を取得します

In [7]:
import urllib.parse
from urllib.error import HTTPError

url1 = "https://data.j-league.or.jp/SFRT01/?competitionSectionIdLabel="
url2 = "&competitionIdLabel="

competitionIdLabel_j1 = ["Ｊリーグ　ディビジョン１", "Ｊリーグ　ディビジョン１", "Ｊリーグ　ディビジョン１", "明治安田生命Ｊ１リーグ １ｓｔ", "明治安田生命Ｊ１リーグ ２ｎｄ", "明治安田生命Ｊ１リーグ １ｓｔ", "明治安田生命Ｊ１リーグ ２ｎｄ", "明治安田生命Ｊ１リーグ", "明治安田生命Ｊ１リーグ", "明治安田生命Ｊ１リーグ", "明治安田生命Ｊ１リーグ", "明治安田生命Ｊ１リーグ"]
competitionIdLabel_j2 = ["Ｊリーグ　ディビジョン２", "Ｊリーグ　ディビジョン２", "Ｊリーグ　ディビジョン２", "明治安田生命Ｊ２リーグ", "明治安田生命Ｊ２リーグ", "明治安田生命Ｊ２リーグ", "明治安田生命Ｊ２リーグ", "明治安田生命Ｊ２リーグ", "明治安田生命Ｊ２リーグ", "明治安田生命Ｊ２リーグ"]
competitionIdLabel_j3 = ["明治安田生命Ｊ３リーグ", "明治安田生命Ｊ３リーグ", "明治安田生命Ｊ３リーグ", "明治安田生命Ｊ３リーグ", "明治安田生命Ｊ３リーグ", "明治安田生命Ｊ３リーグ", "明治安田生命Ｊ３リーグ", "明治安田生命Ｊ３リーグ"]
competitionId_j1 = ["322", "347", "372", "397", "398", "411", "412", "428", "444", "460", "477", "492"]
competitionId_j2 = ["323", "348", "373", "400", "413", "429", "445", "467", "478", "493"]
competitionId_j3 = ["380", "399", "414", "430", "446", "468", "479", "494"]
url3 = "&yearIdLabel="
url4 = "&yearId="
url5 = "&competitionId="
url6 = "&competitionSectionId="
url7 = "&search=search"
df_result = None
df_result_has_stage = None
for year in range(2012, 2022):
    year_id = urllib.parse.quote(str(year)+"年")
    for section in range(1, 50):
        section_Id = urllib.parse.quote("第"+str(section)+"節")
        is_ok_j1_1 = False
        is_ok_j1_2 = False
        is_ok_j2 = False
        is_ok_j3 = False
        print("処理中 :", year, section)
        # J1
        for i in range(len(competitionIdLabel_j1)):
            competition_Id = urllib.parse.quote(competitionIdLabel_j1[i], safe='　')
            competition_Id = competition_Id.replace(' ', '%E3%80%80')
            url_j1 = url1+section_Id+url2+competition_Id+url3+year_id+url4+str(year)+url5+competitionId_j1[i]+url6+str(section)+url7
            time.sleep(0.5) # スリープ処理
            try:
                data_list = pd.read_html(url_j1)
                df_data = pd.DataFrame(data_list[0]) # データフレームに変換
                df_data["year"] = year
                df_data["カテゴリ"] = "J1"
                df_data["節"] = section
                
                comLabel = competitionIdLabel_j1[i]
                if comLabel == "明治安田生命Ｊ１リーグ １ｓｔ":
                    is_ok_j1_1 = True
                    df_data["stage"] = "１ｓｔ"
                    df_result_has_stage = pd.concat([df_result_has_stage, df_data])
                elif comLabel == "明治安田生命Ｊ１リーグ ２ｎｄ":
                    is_ok_j1_2 = True
                    df_data["stage"] = "2ｓｔ"
                    df_result_has_stage = pd.concat([df_result_has_stage, df_data])
                else:
                    is_ok_j1_1 = True
                    is_ok_j1_2 = True
                    df_result = pd.concat([df_result, df_data])
            except HTTPError:
                continue
            
        # J2
        for i in range(len(competitionIdLabel_j2)):
            competition_Id = urllib.parse.quote(competitionIdLabel_j２[i], safe='　')
            competition_Id = competition_Id.replace(' ', '%E3%80%80')
            url_j2 = url1+section_Id+url2+competition_Id+url3+year_id+url4+str(year)+url5+competitionId_j2[i]+url6+str(section)+url7
            time.sleep(0.5) # スリープ処理
            try:
                data_list = pd.read_html(url_j2)
                df_data = pd.DataFrame(data_list[0]) # データフレームに変換
                df_data["year"] = year
                df_data["カテゴリ"] = "J2"
                df_data["節"] = section
                df_result = pd.concat([df_result, df_data])
                is_ok_j2 = True
            except HTTPError:
                continue
        
        # J3
        for i in range(len(competitionIdLabel_j3)):
            competition_Id = urllib.parse.quote(competitionIdLabel_j3[i])
            url_j3 = url1+section_Id+url2+competition_Id+url3+year_id+url4+str(year)+url5+competitionId_j3[i]+url6+str(section)+url7
            time.sleep(0.5) # スリープ処理
            try:
                data_list = pd.read_html(url_j3)
                df_data = pd.DataFrame(data_list[0]) # データフレームに変換
                df_data["year"] = year
                df_data["カテゴリ"] = "J3"
                df_data["節"] = section
                df_result = pd.concat([df_result, df_data])
                is_ok_j3 = True
            except HTTPError:
                continue

処理中 : 2012 1
処理中 : 2012 2
処理中 : 2012 3
処理中 : 2012 4
処理中 : 2012 5
処理中 : 2012 6
処理中 : 2012 7
処理中 : 2012 8
処理中 : 2012 9
処理中 : 2012 10
処理中 : 2012 11
処理中 : 2012 12
処理中 : 2012 13
処理中 : 2012 14
処理中 : 2012 15
処理中 : 2012 16
処理中 : 2012 17
処理中 : 2012 18
処理中 : 2012 19
処理中 : 2012 20
処理中 : 2012 21
処理中 : 2012 22
処理中 : 2012 23
処理中 : 2012 24
処理中 : 2012 25
処理中 : 2012 26
処理中 : 2012 27
処理中 : 2012 28
処理中 : 2012 29
処理中 : 2012 30
処理中 : 2012 31
処理中 : 2012 32
処理中 : 2012 33
処理中 : 2012 34
処理中 : 2012 35
処理中 : 2012 36
処理中 : 2012 37
処理中 : 2012 38
処理中 : 2012 39
処理中 : 2012 40
処理中 : 2012 41
処理中 : 2012 42
処理中 : 2012 43
処理中 : 2012 44
処理中 : 2012 45
処理中 : 2012 46
処理中 : 2012 47
処理中 : 2012 48
処理中 : 2012 49
処理中 : 2013 1
処理中 : 2013 2
処理中 : 2013 3
処理中 : 2013 4
処理中 : 2013 5
処理中 : 2013 6
処理中 : 2013 7
処理中 : 2013 8
処理中 : 2013 9
処理中 : 2013 10
処理中 : 2013 11
処理中 : 2013 12
処理中 : 2013 13
処理中 : 2013 14
処理中 : 2013 15
処理中 : 2013 16
処理中 : 2013 17
処理中 : 2013 18
処理中 : 2013 19
処理中 : 2013 20
処理中 : 2013 21
処理中 : 2013 22
処理中 : 2013 23
処理中 : 2013

In [10]:
df_result.info() # 17882

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16715 entries, 0 to 19
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   グラフ          16715 non-null  object 
 1   順位           16715 non-null  object 
 2   チーム          16715 non-null  object 
 3   勝点           16715 non-null  int64  
 4   試合           16715 non-null  int64  
 5   勝            16715 non-null  int64  
 6   分            16715 non-null  int64  
 7   敗            16715 non-null  int64  
 8   得点           16715 non-null  int64  
 9   失点           16715 non-null  int64  
 10  得失点差         16715 non-null  int64  
 11  直近試合の勝敗      0 non-null      float64
 12  Unnamed: 12  1327 non-null   object 
 13  Unnamed: 13  16715 non-null  int64  
 14  year         16715 non-null  int64  
 15  カテゴリ         16715 non-null  object 
 16  節            16715 non-null  int64  
dtypes: float64(1), int64(11), object(5)
memory usage: 2.3+ MB


In [8]:
df_result.to_csv("j_rank_table_original.csv")

In [9]:
df_result_has_stage.to_csv("j_rank_table_has_st_original.csv")

In [11]:
df_result_has_stage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1224 entries, 0 to 17
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   グラフ          1224 non-null   object 
 1   順位           1224 non-null   int64  
 2   チーム          1224 non-null   object 
 3   勝点           1224 non-null   int64  
 4   試合           1224 non-null   int64  
 5   勝            1224 non-null   int64  
 6   分            1224 non-null   int64  
 7   敗            1224 non-null   int64  
 8   得点           1224 non-null   int64  
 9   失点           1224 non-null   int64  
 10  得失点差         1224 non-null   int64  
 11  直近試合の勝敗      0 non-null      float64
 12  Unnamed: 12  0 non-null      float64
 13  Unnamed: 13  612 non-null    float64
 14  year         1224 non-null   int64  
 15  カテゴリ         1224 non-null   object 
 16  節            1224 non-null   int64  
 17  stage        1224 non-null   object 
 18  年間勝点順位       612 non-null    float64
 19  総勝点     

-57